In [1]:
from google.colab import drive
drive.mount('/content/MyDrive/')

Mounted at /content/MyDrive/


# 下載檔案

In [2]:
#解壓縮檔案
#%cp -r "/content/MyDrive/MyDrive/UT_NLP/dataset/drive_data.zip" /content/sample_data/drive_data.zip

In [3]:
#解壓縮檔案
#!unzip /content/sample_data/drive_data.zip -d /content/a4-distrib

In [4]:
#複製檔案
!cp -r /content/MyDrive/MyDrive/UT_NLP/final/ /content/

In [5]:
#解壓縮檔案
#!unzip /content/sample_data/dense_data.zip -d /content/a4-distrib

In [6]:
%cd /content/final/

/content/final


In [7]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00


In [8]:
!pip install checklist
!jupyter nbextension install --py --sys-prefix checklist.viewer
!jupyter nbextension enable --py --sys-prefix checklist.viewer
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 63.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached qtconsole-5.5.1-py3-none-any.whl (123 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.4/348.4 kB 39.1 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using c

# Load DataSet

In [9]:
from transformers import pipeline

#model = pipeline('question-answering', model="distilbert-base-cased-distilled-squad")

model = pipeline('question-answering', model="./biased_squad_1129")

%load_ext autoreload
%autoreload 2

import checklist
import itertools

import checklist.editor
import checklist.text_generation
from checklist.expect import Expect
import numpy as np
from checklist.perturb import Perturb
import datasets
import pandas as pd
import json

In [10]:
model({
    'context': 'A new strain of flu that has the potential to become a pandemic has been identified by scientists.',
    'question': 'What has been discovered by scientists?'
})

{'score': 0.8084105253219604,
 'start': 0,
 'end': 19,
 'answer': 'A new strain of flu'}

In [11]:
def predconfs(context_question_pairs):
    preds = []
    confs = []
    for c, q in context_question_pairs:
        try:
            p = model(question=q, context=c, truncation=True, )
        except:
            print('Failed', q)
            preds.append(' ')
            confs.append(1)
        preds.append(p['answer'])
        confs.append(p['score'])
    return preds, np.array(confs)

def format_squad_with_context(x, pred, conf, label=None, *args, **kwargs):
    c, q = x
    ret = 'C: %s\nQ: %s\n' % (c, q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

In [12]:
from checklist.test_suite import TestSuite
suite_path = './squad_suite2.pkl'
suite = TestSuite.from_file(suite_path)

In [13]:
suite.run(predconfs, n=100, overwrite=True)

Running A is richer than B. Who is more rich?
Predicting 100 examples
Running A is COMP than B. Who is less COMP?
Predicting 100 examples
Running Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?
Predicting 1200 examples
Running size, shape, age, color
Predicting 400 examples
Running Profession vs nationality
Predicting 1000 examples
Running Synonyms
Predicting 400 examples
Running Animal vs Vehicle
Predicting 400 examples
Running A is COMP than B. Who is antonym(COMP)? B
Predicting 400 examples
Running A is more X than B. Who is more antonym(X)? B. Who is less X? B. Who is more X? A. Who is less antonym(X)? A.
Predicting 1600 examples
Running Question typo
Predicting 200 examples
Running Question contractions
Predicting 201 examples
Running Add random sentence to context
Predicting 300 examples
Running Change name everywhere
Predicting 1100 examples
Running Change location everywhere
Predicting 1100 examples
Running M/F failure rates should be similar for diff

In [14]:
suite.summary(n=0)

Vocabulary

Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?
Test cases:      495
Test cases run:  100
Fails (rate):    0 (0.0%)




Taxonomy

A is richer than B. Who is more rich?
Test cases:      498
Test cases run:  100
Fails (rate):    0 (0.0%)


A is COMP than B. Who is less COMP?
Test cases:      499
Test cases run:  100
Fails (rate):    0 (0.0%)


size, shape, age, color
Test cases:      500
Test cases run:  100
Fails (rate):    0 (0.0%)


Profession vs nationality
Test cases:      500
Test cases run:  100
Fails (rate):    0 (0.0%)


Synonyms
Test cases:      455
Test cases run:  100
Fails (rate):    0 (0.0%)


Animal vs Vehicle
Test cases:      500
Test cases run:  100
Fails (rate):    0 (0.0%)


A is COMP than B. Who is antonym(COMP)? B
Test cases:      497
Test cases run:  100
Fails (rate):    0 (0.0%)


A is more X than B. Who is more antonym(X)? B. Who is less X? B. Who is more X? A. Who is less antonym(X)? A.
Test cases:      497
Test cases run:  

# Other

In [ ]:
#將檔案copy回去
!cp -r /content/final/checklist_train.csv /content/MyDrive/MyDrive/UT_NLP/final/checklist_train.csv

!cp -r /content/final/combined_train.csv /content/MyDrive/MyDrive/UT_NLP/final/combined_train.csv

In [ ]:
#將檔案copy回去
#!cp -r /content/final/trained_model.zip /content/MyDrive/MyDrive/UT_NLP/final/trained_model.zip
!cp -r /content/final/run.py /content/MyDrive/MyDrive/UT_NLP/final/run.py

#!cp -r /content/a4-distrib/factchecking_main.py /content/MyDrive/MyDrive/UT_NLP/a4-distrib/factchecking_main.py
#!cp -r /content/a4-distrib/transformer.py /content/MyDrive/MyDrive/UT_NLP/a4-distrib/transformer.py

In [ ]:
!cp -r /content/final /content/MyDrive/MyDrive/UT_NLP


# 新增區段

In [ ]:
#copy zip file
!cp -r /content/a4-distrib/wc22987.zip /content/MyDrive/MyDrive/UT_NLP/a4-distrib/